In [8]:
from datasets import load_dataset

DATASET_NAME = "qasc"
dataset = load_dataset("allenai/qasc", "", split=["train", "validation"])

In [9]:
choice_len = max([len(x["label"]) for x in dataset[1]["choices"]])
choice_len

8

In [10]:
NUM_PERMUTE_QUESTION = 100
EACH_HAS = 16
NUM_CHOICE=8

In [11]:
# all_ques1=[x for x in dataset[0]['id']]
# all_ques2=[x for x in dataset[1]['id']]

In [12]:
# [x for x in all_ques1 if x in all_ques2]

In [13]:
import numpy as np


def get_prompt(dataset, index, location=-1, has_choice=False):
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        answer_key = json_line["answerKey"]
        answer_key_idx = ord(answer_key[0]) - ord("A")
        answer_text = choices["text"][answer_key_idx]
        if location>-1:
            perm=np.random.permutation(len(choice_texts)-1)
            choice_texts.remove(answer_text)
            choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
            choice_texts.insert(location, answer_text)
        else:
            perm=list(range(NUM_CHOICE))
            choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        
        # print(answer_key_idx,answer_text)
        fact = f"{json_line['combinedfact']}{'' if json_line['combinedfact'][-1]=='.' else '.'}"
        prompt = f"{fact} {question} \\n {candidates}"
        return prompt, answer_text
    except:
        print("QUESTION: ", question, "ANSWER KEY", answer_key)

In [14]:
import random
from tqdm import tqdm, trange
import pickle

if NUM_PERMUTE_QUESTION:
    container = []
    each_choice_has=EACH_HAS/NUM_CHOICE
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        for idx in range(EACH_HAS):
            location=int(idx/each_choice_has)
            content=get_prompt(dataset[1], ques_index, location, has_choice=False)
            # print(content, location)
            container.append(content)
            
    pickle.dump(
        (NUM_PERMUTE_QUESTION, EACH_HAS, container),
        open(f"{DATASET_NAME}_test_permute.pkl", "wb"),
    )
else:
    for index, dataname in enumerate(["train", "test"]):
        container = []
        for ques_index, ques in enumerate(tqdm(dataset[index])):
            container.append(get_prompt(dataset[index], ques_index, has_choice=False))
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 100/100 [00:00<00:00, 634.36it/s]
